In [ ]:
!pip install colossalai

## BASELINE

In [1]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm



class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':30}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

#exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)

# lr_scheduler
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lrs)

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

[04/01/22 13:19:08] INFO     colossalai -                             ]8;id=550767;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=93009;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:19:08,839 INFO: Added key:                        
                             store_based_barrier_key:1 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=872731;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=82092;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:19:08,847 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:1 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=389510;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=795673;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:19:08,860 INFO: Added key:                        
                             store_based_barrier_key:2 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=842502;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=851125;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:19:08,864 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:2 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=296733;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=518696;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:19:08,875 INFO: Added key:                        
                             store_based_barrier_key:3 to store for                          
                             rank: 0                                                         

...

                    INFO     colossalai - colossalai - 2022-04-01 13:26:03,748  ]8;id=370214;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=212953;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Train]: Train-epoch: last =                   
                             11.876 s, mean = 11.876 s | Train-step: last =                  
                             0.013564 s, mean = 0.022073 s | #steps/epoch = 469              

[Epoch 29 / Test]: 100%|██████████| 79/79 [00:01<00:00, 43.95it/s]


[04/01/22 13:26:05] INFO     colossalai - colossalai - 2022-04-01 13:26:05,692  ]8;id=841626;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=531994;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Loss = 76.274                          

                    INFO     colossalai - colossalai - 2022-04-01 13:26:05,706  ]8;id=349849;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=347695;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Test-epoch: last = 1.8039              
                             s, mean = 1.8039 s | Test-step: last = 0.0027525                
                             s, mean = 0.021285 s                                            

                    INFO     colossalai - colossalai - 2022-04-01 13:26:05,721  ]8;id=162876;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=240821;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: GPU: allocated 0.72 MB,                
                             max allocated 35.48 MB, cached: 64.0 MB, max                    
                             cached: 64.0 MB                                                 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

## Test Case 1

- Optimizer - SGD
- LR Scheduler - OneCycleLR

In [1]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm



class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':30}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.12, momentum=0.9, weight_decay=5e-4)

#exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)

# lr_scheduler
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.37, steps_per_epoch=len(train_dataloader), epochs=gpc.config.NUM_EPOCHS)

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

[04/01/22 13:30:20] INFO     colossalai -                             ]8;id=939245;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=126609;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:30:20,817 INFO: Added key:                        
                             store_based_barrier_key:1 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=917693;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=900233;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:30:20,831 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:1 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=896871;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=541087;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:30:20,839 INFO: Added key:                        
                             store_based_barrier_key:2 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=582318;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=215807;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:30:20,852 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:2 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=763738;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=554222;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:30:20,860 INFO: Added key:                        
                             store_based_barrier_key:3 to store for                          
                             rank: 0                                                         

...

                    INFO     colossalai - colossalai - 2022-04-01 13:37:05,902  ]8;id=370214;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=212953;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Train]: Train-epoch: last =                   
                             11.36 s, mean = 11.36 s | Train-step: last =                    
                             0.014933 s, mean = 0.021445 s | #steps/epoch = 469              

[Epoch 29 / Test]: 100%|██████████| 79/79 [00:01<00:00, 48.18it/s]


[04/01/22 13:37:07] INFO     colossalai - colossalai - 2022-04-01 13:37:07,710  ]8;id=841626;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=531994;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Loss = 0.023155                        

                    INFO     colossalai - colossalai - 2022-04-01 13:37:07,716  ]8;id=349849;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=347695;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Test-epoch: last = 1.6418              
                             s, mean = 1.6418 s | Test-step: last = 0.0011935                
                             s, mean = 0.01962 s                                             

                    INFO     colossalai - colossalai - 2022-04-01 13:37:07,733  ]8;id=162876;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=240821;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: GPU: allocated 0.72 MB,                
                             max allocated 35.48 MB, cached: 64.0 MB, max                    
                             cached: 64.0 MB                                                 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

## Test Case 2
- Optimizer: SGD
- LR Scheduler: CosineAnnealingLR

In [1]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm



class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':30}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

#exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)

# lr_scheduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = gpc.config.NUM_EPOCHS, eta_min=0)

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

[04/01/22 13:52:41] INFO     colossalai -                             ]8;id=72543;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=642439;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:52:41,297 INFO: Added key:                        
                             store_based_barrier_key:1 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=839323;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=458454;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:52:41,306 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:1 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=556617;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=644612;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:52:41,314 INFO: Added key:                        
                             store_based_barrier_key:2 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=489519;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=296536;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:52:41,323 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:2 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=644705;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=950175;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-04-01 13:52:41,327 INFO: Added key:                        
                             store_based_barrier_key:3 to store for                          
                             rank: 0                                                         

...

                    INFO     colossalai - colossalai - 2022-04-01 13:59:30,045  ]8;id=370214;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=212953;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Train]: Train-epoch: last =                   
                             11.871 s, mean = 11.871 s | Train-step: last =                  
                             0.011493 s, mean = 0.021747 s | #steps/epoch = 469              

[Epoch 29 / Test]: 100%|██████████| 79/79 [00:01<00:00, 44.23it/s]


[04/01/22 13:59:31] INFO     colossalai - colossalai - 2022-04-01 13:59:31,993  ]8;id=841626;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=531994;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Loss = 0.034936                        

[04/01/22 13:59:32] INFO     colossalai - colossalai - 2022-04-01 13:59:32,005  ]8;id=349849;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=347695;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: Test-epoch: last = 1.7867              
                             s, mean = 1.7867 s | Test-step: last = 0.0013006                
                             s, mean = 0.020957 s                                            

                    INFO     colossalai - colossalai - 2022-04-01 13:59:32,017  ]8;id=162876;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=240821;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: [Epoch 29 / Test]: GPU: allocated 0.72 MB,                
                             max allocated 35.48 MB, cached: 64.0 MB, max                    
                             cached: 64.0 MB                                                 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs